In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Params_PLD.jl")
include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
include("Estimation_PLD_Fast.jl")
include("Models_PLD.jl")
include("Evaluation_PLD.jl")
include("Implement_All_Methods_PLD.jl")
include("Figures_PLD.jl")

hist_profit_distribution (generic function with 1 method)

In [2]:
Params = get_default_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_train = Params["S_train"] # training data size
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_this = Params["coef_this"] # coefficient for data generation
coef_Wang_Qi_Shen = Params["coef_Wang_Qi_Shen"] # coefficient for Wang, Qi, Shen data generation

(alp0_lb = 1.0, alp0_ub = 2.0, alp_lb = -1.0, alp_ub = 1.0, beta_lb = -2.0, beta_ub = 2.0, A_lb = -2.0, A_ub = 2.0, r0_lb = 0.0, r0_ub = 1.0, r_lb = -1.0, r_ub = 1.0)

In [3]:
N_x = 2

2

In [4]:
Random.seed!(seed)
is_Wang_Qi_Shen = false;
is_same_util_para = true;
if is_Wang_Qi_Shen
    coef_para_Input = coef_Wang_Qi_Shen;
    project_dir = "Wang_Qi_Shen_N=$(N)_N_x=$(N_x)_N_u=$(N_u)_S_train=$(S_train)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
else
    coef_para_Input = coef_this;
    project_dir = "N=$(N)_N_x=$(N_x)_N_u=$(N_u)_S_train=$(S_train)_N_nonzero=$(N_nonzero)_dr=$(d_r[1])_seed=$(seed)"
end
if is_same_util_para
    println("Generate data with the same utility parameters for all instances.")
    theta_true_Fixed, r_params_Fixed = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero,coef_para_Input);
    project_dir = string(project_dir, "_Same_Util_Para/")
else
    println("Generate data with different utility parameters for all instances.")
    project_dir = string(project_dir, "_Diff_Util_Para/")
end
current_dir = pwd()
parent_dir = dirname(current_dir)
grand_pa_dir = dirname(parent_dir)
data_dir = string(dirname(grand_pa_dir), "/Data/Product_Line_Design/")
data_dir = string(data_dir,project_dir)
if !isdir(data_dir)
    mkpath(data_dir)
end
println("Data directory: ", data_dir)
save(string(data_dir, "Params.jld2"), Params);

Generate data with the same utility parameters for all instances.
Data directory: /Users/zhangxun/Dropbox/Research/Robust_Exp/Data/Product_Line_Design/N=3_N_x=2_N_u=1_S_train=100_N_nonzero=5_dr=2.0_seed=1_Same_Util_Para/


In [5]:
function compute_w(params,z_input)
    alpha0 = params.alpha0
    alpha = params.alpha
    beta = params.beta
    A = params.A
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [6]:
S_train = 100

100

In [7]:
Input_Data_this = Generate_Data_this_Same_Para(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed);
theta_true,r_params,X_train,Y_train,Z_train,asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);

In [8]:
is_ridge = false
lbd = 0.0

0.0

In [9]:
alp_0_exp, alpha_exp, beta_exp, A_exp, solve_time, sol_status, obj_val = Estimate_OPT_Model_PLD(N_Max,N_x,N_u,Y_train,X_train,Z_train, asorrtment_train,is_ridge, lbd);

In [11]:
theta_Exp = (alpha0 = alp_0_exp,
              alpha = alpha_exp,
              beta = beta_exp,
              A = A_exp)
nu0_Exp,nu_Exp = compute_w(theta_Exp,Z_test[1,:]);
nu0_true,nu_true = compute_w(theta_true,Z_test[1,:]);

# theta_hat = Estimation_This(0.0,X_train,Y_train,Z_train,N_x,N_u)
alpha0_hat, alpha_hat, beta_hat, A_hat, opt_result, X_tilde = estimate_parameters_fast(X_train, Y_train, Z_train, 0.0, N_x, N_u);
theta_hat = (alpha0=alpha0_hat, alpha=alpha_hat, beta=beta_hat, A=A_hat)
nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,:])  

println("nu0 true = ",round(nu0_true,digits=4),", nu_true = ", round.(nu_true,digits=4))
println("nu0 hat = ",round(nu0_hat,digits=4),", nu_hat = ", round.(nu_hat,digits=4))
println("nu0 Exp = ",round(nu0_Exp,digits=4),", nu_Exp = ", round.(nu_Exp,digits=4))

  0.000051 seconds (204 allocations: 32.266 KiB)
nu0 true = 0.0151, nu_true = [-0.6411, -0.2919]
nu0 hat = -0.3428, nu_hat = [0.3644, -0.9148]
nu0 Exp = -0.3428, nu_Exp = [0.3644, -0.9148]
